# Making malls

## Local stores

In [1]:
from dol import JsonFiles, wrap_kvs, mk_dirs_if_missing, temp_dir, KeyPath, KeyTemplate, filt_iter, Files
import os

pjoin = os.path.join

In [2]:
# temporary directory

rootdir = temp_dir('ragdag/rjsf_fiddle')

print(rootdir)

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle


In [4]:
## Making a key template

from dol import KeyTemplate
import os

pjoin = os.path.join

user_template = pjoin('u', '{user_name}')
stores_template = pjoin('stores', '{store_name}')

kt = KeyTemplate(template=pjoin(user_template, stores_template))
key_wrap = kt.key_codec(decoded='tuple', encoded='str')

back = {
    'u/bob/stores/spec_1': {'an': 'apple'},
    'u/bob/stores/spec_2': {'an': 'orange'},
    'u/alice/stores/spec_1': {'i_like': 'to move it'},
    'some/thing/else': 'contents'
}

front = key_wrap(kt.filt_iter(back))
assert sorted(front) == [('alice', 'spec_1'), ('bob', 'spec_1'), ('bob', 'spec_2')]
front['bob', 'spec_1']


{'an': 'apple'}

In [7]:
from raglab.stores.stores_util import *

def test_mk_local_user_store(verbose=False):

    rootdir = temp_dir('ragdag/rjsf_fiddle')
    if verbose:
        print(f"{rootdir=}")

    s = mk_local_user_store(rootdir, user_name='bob', store_kind='spec_1')
    s['some_key'] = b'some_value'
    s['some_other_key'] = b'some_other_value'
    if verbose:
        print(f"Go check out this folder: file://{s.rootdir}")
    assert 'some_key' in s
    assert s['some_key'] == b'some_value'
    assert 'some_key' in os.listdir(s.rootdir)

def test_mk_specialized_local_user_store(verbose=False):

    rootdir = temp_dir('ragdag/rjsf_fiddle')
    if verbose:
        print(f"{rootdir=}")

    s = mk_json_store(rootdir, user_name='alice', store_kind='jsons')
    s['some_key'] = {'a': 'b'}
    assert s['some_key'] == {'a': 'b'}

    # see that the file is there, with a json extension
    assert 'some_key.json' in os.listdir(s.rootdir)
    # see that the file contains the expected content, as json string
    from pathlib import Path
    assert Path(os.path.join(s.rootdir, 'some_key.json')).read_text() == '{"a": "b"}'

    s = mk_text_store(rootdir, user_name='alice', store_kind='texts')
    s['some_key'] = 'some_value'
    assert s['some_key'] == 'some_value'

    s = mk_pickle_store(rootdir, user_name='alice', store_kind='pickles')
    ecclectic_data = [1, 2.3, 'four']
    s['some_key'] = ecclectic_data
    assert s['some_key'] == ecclectic_data

test_mk_local_user_store(verbose=True)
test_mk_specialized_local_user_store()


rootdir='/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle'
Go check out this folder: file:///var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle/u/bob/stores/spec_1/


In [46]:
rootdir

'/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ragdag/rjsf_fiddle'

In [43]:
list(s)
s['some_key']

b'some_value'

In [44]:
s = JsonFiles(rootdir)
list(s)

[]

In [24]:
s['asdf.json'] = {'a': 'b'}
list(s)

['asdf', 'some_key', 'Nonesome_key.json', 'asdf.json']

In [29]:
w = KeyCodecs.affixed(prefix='', suffix='.json')
ss = w({'a.json': {'a': 'b'}})
list(ss)

['a']